### Importing Libraries
This cell imports necessary libraries for the project:
- **PyTorch (`torch`)**: The main library for building and training neural networks.
- **Submodules (`nn`, `optim`)**: For defining network layers and optimization algorithms.
- **Torchvision**: For handling image data, including datasets, transformations, and models.
- **Matplotlib**: For plotting and visualizing data.
- **NumPy**: For numerical operations.

In [1]:
# Importing necessary libraries for PyTorch, dataset handling, and visualization
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models

### Setting the Device
This cell defines the device (GPU or CPU) to be used for training the neural network. If a CUDA-capable GPU is available, it's used for faster computation; otherwise, the CPU is used.

In [2]:
# Define the device for training the model (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Data Transformations
This cell sets up the transformations for the input images. The transformations include resizing images to a specific size, converting them to tensor format (which is required by PyTorch), and normalizing the pixel values.

In [3]:
# Define transformations for the input data
# These transformations are used for data augmentation and preprocessing.
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # Randomly flip the input horizontally (data augmentation).
    transforms.ToTensor() # Convert the data to PyTorch tensors
])

### Loading the Dataset
This cell loads the MNIST dataset, which consists of 60,000 28x28 grayscale images of handwritten digits in 10 classes (0-9). The dataset is divided into training and testing sets to facilitate machine learning model development. Data loaders are also defined for iterating over these sets in batches.

In [4]:
# Load MNIST dataset (training and testing) with defined transformations
# - 'train=True' loads the training dataset.
# - 'train=False' loads the testing dataset.
# - 'download=True' downloads the dataset if not already downloaded.
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Define data loaders for training and testing sets
# - 'batch_size' determines the number of samples in each batch.
# - 'shuffle=True' shuffles the training data to improve model training.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 107624613.95it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 118644166.33it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 42710724.76it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 19823651.16it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# Create a ResNet-50 model pre-trained
resnet = models.resnet50(pretrained=True)
# Check the ResNet-50 model
resnet

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 49.8MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# Modify the first convolutional layer to accept grayscale images (1 channel)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the fully connected layer (classifier) for the number of output classes
resnet.fc = nn.Linear(2048, 10, bias=True)

#Check the modified ResNet-50 model
resnet

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
# Move the model to the specified device (e.g., GPU if available)
resnet.to(device)

# Define the loss function (CrossEntropyLoss) for classification
criterion = nn.CrossEntropyLoss()

# Define the optimizer (Stochastic Gradient Descent - SGD) for model parameters
optimizer = optim.SGD(resnet.parameters(), lr=0.01, momentum=0.9)

### Training the Model
This part of the code is responsible for training the neural network. It involves the following key steps:
- **Setting Number of Epochs**: `num_epochs` is defined to specify the number of times the entire dataset is passed through the network.
- **Training Loop**: For each epoch, the code iterates over the training data in mini-batches. In each iteration:
  - The data is moved to the specified device (GPU or CPU).
  - Gradients are reset using `optimizer.zero_grad()`.
  - A forward pass is performed to compute the output.
  - Loss is computed and backpropagation is performed.
  - The optimizer updates the model parameters.
  - The running loss is calculated and printed every 100 mini-batches for monitoring.

### Testing the Model and Calculating Accuracy
After each training epoch, the model is evaluated on the test dataset:
- The gradients are not computed in this phase (`torch.no_grad()`).
- The model makes predictions on the test dataset and the accuracy is calculated by comparing these predictions with the true labels.
- The accuracy is stored in `accuracy_history` for visualization.

In [ ]:
# Train the model
num_epochs = 5  # Number of training epochs
accuracy_history = []  # To store accuracy history during training

# Loop over the specified number of epochs
for epoch in range(num_epochs):
    running_loss = 0.0  # Initialize the running loss for this epoch

    # Loop over mini-batches of training data
    for i, data in enumerate(train_loader, 1):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients to avoid accumulation
        optimizer.zero_grad()

        # Forward pass: Compute the predictions and loss
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)

        # Backward pass: Compute gradients and update model parameters
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print('[Epoch %d, Batch %5d] Loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    # Test the model and calculate accuracy
    correct = 0  # Initialize the number of correctly predicted samples
    total = 0    # Initialize the total number of samples

    with torch.no_grad():
        # Loop over mini-batches of test data
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass: Compute the model's predictions
            outputs = resnet(images)

            # Use the "torch.max" function to get the index of the maximum predicted value
            _, predicted = torch.max(outputs.data, 1)

            # Update the total count of samples
            total += labels.size(0)

            # Count the number of correctly predicted samples
            correct += (predicted == labels).sum().item()

    # Calculate the accuracy as the percentage of correctly predicted samples
    accuracy = 100 * correct / total

    # Append the accuracy to the accuracy history list
    accuracy_history.append(accuracy)

    # Print the accuracy of the network on the test images
    print('Accuracy of the network on the test images: %d %%' % accuracy)

[Epoch 1, Batch   100] Loss: 0.700
[Epoch 1, Batch   200] Loss: 0.200
[Epoch 1, Batch   300] Loss: 0.122
[Epoch 1, Batch   400] Loss: 0.115
Accuracy of the network on the test images: 97 %


### Plotting the Accuracy Over Epochs
This section plots the accuracy of the model over the training epochs:
- A line plot is created using `matplotlib` to visualize the accuracy over epochs.
- This helps in understanding the learning progress and diagnosing issues like overfitting or underfitting.

In [ ]:
# Plot the accuracy over epochs
plt.plot(accuracy_history)  # Plot the accuracy values stored in accuracy_history
plt.title('Accuracy over Epochs')  # Set the title of the plot
plt.xlabel('Epoch')  # Label for the x-axis (epochs)
plt.ylabel('Accuracy (%)')  # Label for the y-axis (accuracy percentage)
plt.show()  # Display the plot